In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import os

target_dir = "../"
zip_file_path = "generated.zip"

if not os.path.exists(target_dir + 'generated'):
    print(f"The directory {target_dir} does not exist. Proceeding with download.")

    !apt-get update
    !apt-get install unzip
    
    !curl "https://drive.usercontent.google.com/download?id=10xaugPOoC3SraTwp90sfqSMYQEQK96Ls&confirm=xxx" -o {zip_file_path}
    !mkdir -p {target_dir}
    
    !unzip {zip_file_path} -d {target_dir}
    
    print(f"File downloaded and extracted to {target_dir}")
    
    !rm {zip_file_path}
else:
    print(f"The directory {target_dir} already exists. No action taken.")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import sys
from tqdm import tqdm
from datetime import datetime
pd.options.mode.chained_assignment = None

from pathlib import Path

repo_root = Path.cwd().parent.resolve()
sys.path.append(str(repo_root))

from sklearn.model_selection import train_test_split
from src.data.auction_dataset import AuctionDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
pairs = pd.read_csv('../generated/auction_indices.csv')

pairs.head()

,record,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
0,2025-03-11 00:00:00,5089,6,8.00,7.12,0.0,21.0,13.33,12.71,2.0,31.0
1,2025-03-11 00:00:00,12178,3,21.67,17.17,0.0,42.0,12.00,13.59,0.0,31.0
2,2025-03-11 00:00:00,1408,5,3.80,7.11,0.0,18.0,25.00,11.03,3.0,31.0
3,2025-03-11 00:00:00,15771,4,6.75,7.79,0.0,19.0,30.00,1.73,27.0,31.0
4,2025-03-11 00:00:00,15754,4,14.00,10.39,0.0,24.0,27.00,4.00,23.0,31.0


## Prepare and balance data

In [4]:
pd.options.display.float_format = '{:.2f}'.format
pairs.describe()

,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
count,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00
mean,8706.56,6.70,19.28,9.16,8.28,32.08,16.10,7.77,6.89,26.45
std,5085.27,20.71,9.17,5.94,9.94,13.17,8.03,4.90,9.07,9.90
min,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,4251.00,2.00,13.00,4.15,1.00,21.00,10.33,3.27,1.00,22.00
50%,8632.50,4.00,18.67,9.88,4.00,38.00,15.67,9.16,3.00,31.00
75%,13070.00,6.00,24.50,14.00,13.00,44.00,21.25,11.69,8.00,34.00
max,17601.00,692.00,47.00,23.50,47.00,47.00,37.00,18.50,37.00,37.00


In [5]:
pairs = pairs[pairs['g_hours_on_sale_max'] <= 50]
pairs = pairs[pairs['g_current_hours_max'] <= 50]

pairs.describe()

,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
count,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00,100000.00
mean,8706.56,6.70,19.28,9.16,8.28,32.08,16.10,7.77,6.89,26.45
std,5085.27,20.71,9.17,5.94,9.94,13.17,8.03,4.90,9.07,9.90
min,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,4251.00,2.00,13.00,4.15,1.00,21.00,10.33,3.27,1.00,22.00
50%,8632.50,4.00,18.67,9.88,4.00,38.00,15.67,9.16,3.00,31.00
75%,13070.00,6.00,24.50,14.00,13.00,44.00,21.25,11.69,8.00,34.00
max,17601.00,692.00,47.00,23.50,47.00,47.00,37.00,18.50,37.00,37.00


In [6]:
train_pairs, val_pairs = train_test_split(pairs, test_size=0.05, random_state=42, shuffle=False)

print(f"Before filtering: {len(train_pairs)}")

train_pairs = train_pairs[train_pairs['g_hours_on_sale_len'] <= 64]
val_pairs = val_pairs[val_pairs['g_hours_on_sale_len'] <= 64]

print(f"After filtering: {len(train_pairs)}\n")

train_pairs = train_pairs[:int(len(train_pairs)*0.90)]

print(f"Train pairs: {len(train_pairs)}")
print(f"Val pairs: {len(val_pairs)}")

Before filtering: 95000
After filtering: 94305

Train pairs: 84874
Val pairs: 4885


In [7]:
train_pairs.tail()

,record,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
85558,2025-03-11 05:00:00,1501,5,9.20,6.76,1.00,20.00,15.80,12.78,0.00,35.00
85559,2025-03-11 05:00:00,1625,4,11.00,0.00,11.00,11.00,35.00,0.00,35.00,35.00
85560,2025-03-11 05:00:00,9596,4,18.00,4.30,11.00,22.00,10.50,14.19,1.00,35.00
85561,2025-03-11 05:00:00,4603,8,15.00,9.47,2.00,32.00,15.38,9.58,5.00,35.00
85562,2025-03-11 05:00:00,938,4,15.50,14.99,3.00,41.00,13.50,12.54,4.00,35.00


In [8]:
val_pairs.head()

,record,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
95000,2025-03-11 05:00:00,5505,1,21.00,0.00,21.00,21.00,2.00,0.00,2.00,2.00
95001,2025-03-11 05:00:00,8011,3,45.33,0.47,45.00,46.00,1.67,0.47,1.00,2.00
95002,2025-03-11 05:00:00,14658,1,21.00,0.00,21.00,21.00,2.00,0.00,2.00,2.00
95003,2025-03-11 05:00:00,10056,1,21.00,0.00,21.00,21.00,2.00,0.00,2.00,2.00
95004,2025-03-11 05:00:00,8302,2,28.00,17.00,11.00,45.00,1.00,1.00,0.00,2.00


In [ ]:
# histogram of group_hours_on_sale_mean
plt.hist(train_pairs['g_hours_on_sale_mean'], bins=30)

In [ ]:
val_pairs.sample(5)

In [ ]:
plt.hist(train_pairs['g_hours_on_sale_len'], bins=10)
plt.show()

In [ ]:
plt.hist(train_pairs['g_hours_on_sale_mean'], bins=15)
plt.show()

In [9]:
import json
import os

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
    item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

In [10]:
feature_stats = torch.load('../generated/feature_stats.pt')

In [11]:
from src.data.auction_dataset import AuctionDataset
from src.data.utils import collate_auctions

batch_size = 128

train_dataset = AuctionDataset(train_pairs, feature_stats=feature_stats, path='../generated/sequences.h5')
val_dataset = AuctionDataset(val_pairs, feature_stats=feature_stats, path='../generated/sequences.h5')

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_auctions, num_workers=8, prefetch_factor=8, pin_memory=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_auctions, num_workers=8, prefetch_factor=8, pin_memory=True)

## Model definition

In [15]:
test_data_loader = True

if test_data_loader:
    iter_loader = iter(train_dataloader)
    batch = next(iter_loader)
    auctions = batch['auctions']
    item_index = batch['item_index']
    contexts = batch['contexts']
    bonus_lists = batch['bonus_lists']
    modifier_types = batch['modifier_types']
    modifier_values = batch['modifier_values']
    current_hours_raw = batch['current_hours_raw']
    time_left_raw = batch['time_left_raw']
    y = batch['target']

    print(f'auctions: {auctions.shape}')
    print(f'item_index: {item_index.shape}')
    print(f'contexts: {contexts.shape}')
    print(f'bonus_lists: {bonus_lists.shape}')
    print(f'modifier_types: {modifier_types.shape}')
    print(f'modifier_values: {modifier_values.shape}')
    
    print(f'y: {y.shape}')

auctions: torch.Size([128, 56, 9])
item_index: torch.Size([128, 56])
contexts: torch.Size([128, 56])
bonus_lists: torch.Size([128, 56, 7])
modifier_types: torch.Size([128, 56, 7])
modifier_values: torch.Size([128, 56, 7])
y: torch.Size([128, 56])


In [ ]:
from src.models.auction_transformer import AuctionTransformer

input_size = 17
embedding_dim = 64
d_model = 512
dim_feedforward = d_model * 4
nhead = 16
num_layers = 12
dropout_p = 0.1
n_items = len(item_to_idx)
n_contexts = len(context_to_idx) + 1
n_bonuses = len(bonus_to_idx)
n_modtypes = len(modtype_to_idx)

model = AuctionTransformer(
    input_size, 
    n_items, 
    n_contexts,
    n_bonuses,
    n_modtypes,
    embedding_dim, 
    d_model, 
    dim_feedforward, 
    nhead, 
    num_layers,
    dropout_p=dropout_p,
    learning_rate=1e-4,
    logging_interval=500
)

print(sum(p.numel() for p in model.parameters()))

## Training

In [ ]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint

name = 'auction_transformer_40M'

logger = WandbLogger(
    project="auction_transformer",
    name=name
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f'../models/{name}',
    filename='epoch_{epoch:02d}',
    save_top_k=-1,
    every_n_epochs=1,
    save_last=True
)

trainer = L.Trainer(
    max_epochs=1,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    logger=logger,
    limit_val_batches=500,
    val_check_interval=0.25,
    callbacks=[checkpoint_callback]
)

trainer.fit(model, train_dataloader, val_dataloader)